# Data Cleaning for Final Project
##### Julie Lee, Julia Bernstein, Nandan Aggarwal

### Importing

Below, we import pandas, numpy, matplotlib, and seaborn in order to help us with our data cleaning.

In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Firstly, we imported our two datasets from [data.world](https://data.world/kcmillersean/billboard-hot-100-1958-2017).

The first dataset, which we saved under the name "dfHS," was a .csv file named Hot Stuff.csv. This file contains all 100 songs on the Billboard Hot 100 for each week ranging from 8/2/1958 and 12/28/2019.

The other dataset, which we saved under the name "df100," was orginally a .xlsx (Excel) file named Hot 100 Audio Features.xlsx. This file contains characteristics for each song that reached the Billboard Hot 100, pulled directly from Spotify. 

In [135]:
dfHS = pd.read_csv('Hot Stuff.csv')
df100 = pd.read_excel('Hot 100 Audio Features.xlsx')


We limited the songs to those that are not longer than 6 minutes as there are some songs that are quite long and create outliers when examining the data.

In [136]:
df100 = df100[df100['spotify_track_duration_ms']<=360000]

We added a  "Year" column to dfHS because we can manipulate the datetime objects for easier use.

In [137]:
dfHS['Year'] = pd.to_datetime(dfHS['WeekID'], format = '%m/%d/%Y').dt.year

In the next cells, we limit our dataframe to the data in between 2005 and 2015.

In [138]:
dfHS2 = dfHS[dfHS['Year']>=2005]

In [139]:
dfHS2 = dfHS2[dfHS2['Year']<=2015]

We drop unneccesary columnns from dfHS, which we felt would not help us with our research questions.

In [140]:
dfHS2 = dfHS2.drop(columns = ['url', 'WeekID', 'Week Position', 'Instance', 'Previous Week Position'])

We group the Hot Stuff data by 'SongID' to get groups of each occurance of a particular song. For example, if a song charted for 30 weeks, there will be 30 occurances of that song in one group. We then created a new dataframe, minimums. In minimums, we found the index of the grouped song where it reached its peak position, in order to only keep that particular occurance. We also wanted to find the last week the song charted, so that we could find the total number of weeks that the song remained in the Hot 100.

In [141]:
grouped_df = dfHS2.groupby('SongID')
minimums = grouped_df['Peak Position'].idxmin()
mostweeks = grouped_df['Weeks on Chart'].max().to_frame()

In [142]:
dfHS_update = dfHS2.loc[minimums].drop(columns = 'Weeks on Chart')
dfHS_final = dfHS_update.merge(mostweeks, on = 'SongID')

We create an updated dataframe, `dfHS_update`, which only contains each song once (at its peak). We achieved this by creating an array called minimums, which contained the indices of the songs at their peak position (<strong> NOTE: </strong> we use idxmin because in the context of the Billboard chart, a lower number indicates a higher position). 

We then drop the old "weeks on chart" column in `dfHS_update`, which contains the week that a song peaked, which is not necessarily the total amount of weeks that it spent on the Hot 100, therefore making it unnecessary to our research.
- For example, a song might peak in its 6th week, fall to a lower position, but remain on the chart for 10 additional weeks. 

Thus, we wanted to keeep the total number of weeks that a song spent on the chart, which we did by checking the song's max number of weeks on chart, putting this in a new dataframe, and then merging it with the updated dataframe on SongID.

In [143]:
df100_updated = df100.drop(columns = ['Performer', 'Song', 'spotify_genre', 'spotify_track_id', 'spotify_track_preview_url', 
                                      'spotify_track_album', 'spotify_track_explicit', 'key', 'loudness', 
                                      'speechiness', 'instrumentalness', 'liveness', 'mode', 'time_signature'])

We also dropped additional unnecessary or duplicate columns to keep our dataframe concise.

### Merging

We decided to merge "df100" and the updated "dfHS" (dfHS_updated) into one dataset called "final_df." To do so, we matched the two datasets by their "SongID" element, dropped null values and repeated columns, and reset the index. 

In [144]:
final_df = df100_updated.merge(dfHS_final, on = 'SongID')
final_df = final_df.dropna().drop_duplicates('SongID', False).reset_index(drop = True)

In this step, we convert out durations from milliseconds to minutes.

In [145]:
final_df['spotify_track_duration_ms'] = final_df['spotify_track_duration_ms']/60000

We multiply small values (those on a 0 to 1 scale) by 100 for a more accurate data visualization when comparing with other attributes ranked out of 100.

In [146]:
final_df[['danceability', 'energy', 'acousticness', 'valence']] = final_df[['danceability', 'energy', 
                                                                            'acousticness', 'valence']]*100

## Renaming Columns

In [147]:
final_df = final_df.rename(columns={"SongID": "song_id", 
                                    "spotify_track_duration_ms": "duration_mins", 
                                    "spotify_track_popularity": "popularity", 
                                    "Peak Position": "peak_position", 
                                    "Song":"song", "Performer":"performer", 
                                    "Year":"year", "Weeks on Chart":'total_charting_weeks'})

Our final step in our cleaning is adding a true false column for songs that ever broke into the top 10.

In [148]:
final_df['top10'] = final_df['peak_position'] <= 10

In [149]:
final_df.head()

,song_id,duration_mins,popularity,danceability,energy,acousticness,valence,tempo,song,performer,peak_position,year,total_charting_weeks,top10
0,ApplauseLady Gaga,3.538883,71.0,66.9,78.0,2.65,73.8,139.945,Applause,Lady Gaga,4.0,2013,23.0,True
1,Story Of My LifeOne Direction,4.091550,81.0,60.0,66.3,22.50,28.6,121.070,Story Of My Life,One Direction,6.0,2013,32.0,True
2,"Bang BangJessie J, Ariana Grande & Nicki Minaj",3.323100,79.0,70.6,78.6,26.00,74.9,150.035,Bang Bang,"Jessie J, Ariana Grande & Nicki Minaj",3.0,2014,31.0,True
3,"Sorry, Blame It On MeAkon",4.928667,59.0,71.3,64.3,27.60,50.8,176.079,"Sorry, Blame It On Me",Akon,7.0,2007,19.0,True
4,Strange CloudsB.o.B Featuring Lil Wayne,3.771550,61.0,53.1,66.5,6.07,80.0,73.726,Strange Clouds,B.o.B Featuring Lil Wayne,7.0,2011,20.0,True


In [151]:
# Export to csv final for use in project write up
final_df.to_csv('cleaned.csv')